In [1]:
import numpy as np

np.random.seed(42)
sigma = 0.03  # exploration strength

# ----- INPUTS FROM YOUR FILE -----
week1_inputs = [
    np.array([0.333333, 0.666667]),
    np.array([0.666667, 0.333333]),
    np.array([0.25, 0.5, 0.75]),
    np.array([0.2, 0.4, 0.6, 0.8]),
    np.array([0.2, 0.4, 0.6, 0.8]),
    np.array([0.166667, 0.333333, 0.5, 0.666667, 0.833333]),
    np.array([0.142857, 0.285714, 0.428571, 0.571429, 0.714286, 0.857143]),
    np.array([0.111111, 0.222222, 0.333333, 0.444444, 0.555556, 0.666667, 0.777778, 0.888889])
]

week2_inputs = [
    np.array([0.666667, 0.333333]),
    np.array([0.5, 0.5]),
    np.array([0.75, 0.5, 0.25]),
    np.array([0.8, 0.6, 0.4, 0.2]),
    np.array([0.25, 0.45, 0.65, 0.85]),
    np.array([0.833333, 0.666667, 0.5, 0.333333, 0.166667]),
    np.array([0.857143, 0.714286, 0.571429, 0.428571, 0.285714, 0.142857]),
    np.array([0.161111, 0.211111, 0.261111, 0.311111, 0.361111, 0.411111, 0.461111, 0.511111])
]

# ----- OUTPUTS FROM YOUR FILE -----
week1_outputs = np.array([
    5.724814756293084e-48, 0.5775314149002366, -0.10368178770888667,
    -13.072131637188551, 7.608500025600002, -1.3339496662959942,
    0.18787317822666935, 9.0501520348139
])

week2_outputs = np.array([
    5.724814756293084e-48, 0.7807696886511188, -0.11568071819994481,
    -13.072131637188551, 56.23701453439996, -1.3593478122916869,
    0.0798632646683717, 9.6699082341974
])

# ----- FORMATTER -----
def format_query(x):
    return "-".join([f"0.{int(v * 1e6):06d}" for v in x])

# ----- QUERY GENERATION -----
queries = []

for i in range(8):
    # choose better week (maximize)
    base = week2_inputs[i] if week2_outputs[i] > week1_outputs[i] else week1_inputs[i]

    noise = np.random.normal(0, sigma, size=base.shape)
    new_x = np.clip(base + noise, 0.0, 0.999999)

    queries.append(format_query(new_x))

# ----- PRINT RESULTS -----
for i, q in enumerate(queries, 1):
    print(f"Function {i}: {q}")


Function 1: 0.348234-0.662519
Function 2: 0.519430-0.545690
Function 3: 0.242975-0.492975-0.797376
Function 4: 0.223023-0.385915-0.616276-0.786097
Function 5: 0.236028-0.457258-0.592601-0.798252
Function 6: 0.149798-0.302948-0.509427-0.639426-0.790963
Function 7: 0.186826-0.278940-0.430596-0.528686-0.697954-0.860470
Function 8: 0.126581-0.222381-0.243091-0.302360-0.343059-0.466679-0.460706-0.479379


In [3]:
import numpy as np
import pandas as pd
from pathlib import Path
import re

# -----------------------------
# Paths
# -----------------------------
INPUTS_FILE = Path("inputs.txt")
OUTPUTS_FILE = Path("outputs.txt")

# -----------------------------
# 1. Read files line-by-line (one week per line)
# -----------------------------
input_lines = [line.strip() for line in INPUTS_FILE.read_text().splitlines() if line.strip()]
output_lines = [line.strip() for line in OUTPUTS_FILE.read_text().splitlines() if line.strip()]

if len(input_lines) != len(output_lines):
    raise ValueError(f"Week mismatch: inputs={len(input_lines)}, outputs={len(output_lines)}")

n_weeks = len(input_lines)
print("Weeks detected:", n_weeks)

# -----------------------------
# 2. Parse into nested lists: [week][function]
# -----------------------------
inputs_list = []   # [week][function][dim]
outputs_list = []  # [week][function]

for w in range(n_weeks):
    # ---- INPUTS ----
    line_in = input_lines[w]
    arrays = re.findall(r"array\(\[(.*?)\]\)", line_in, re.S)

    week_inputs = []
    for a in arrays:
        nums = [float(x) for x in a.replace("\n", " ").split(",") if x.strip()]
        week_inputs.append(np.array(nums))
    inputs_list.append(week_inputs)

    # ---- OUTPUTS ----
    line_out = output_lines[w]
    nums_out = re.findall(r"[-+]?\d*\.\d+e[-+]?\d+|[-+]?\d*\.\d+|[-+]?\d+", line_out)
    week_outputs = [float(x) for x in nums_out]
    outputs_list.append(week_outputs)

# Sanity checks
n_functions = len(inputs_list[0])
for w in range(n_weeks):
    assert len(inputs_list[w]) == n_functions
    assert len(outputs_list[w]) == n_functions

print("Functions per week:", n_functions)
print("✅ Parsed inputs/outputs successfully")

# -----------------------------
# 3. Helper: format for portal
# -----------------------------
def format_query(x):
    x = np.clip(x, 0.0, 0.999999)
    return "-".join([f"0.{int(v * 1_000_000):06d}" for v in x])

# -----------------------------
# 4. Generate new (4th-round) values
#    Strategy: local exploration around latest week (week - 1)
# -----------------------------
EPS = 0.02
latest_week_idx = n_weeks - 1

new_queries = []

for func_idx in range(n_functions):
    base = inputs_list[latest_week_idx][func_idx]
    new_x = np.clip(base + EPS, 0.0, 0.999999)
    q_str = format_query(new_x)
    new_queries.append(q_str)
    print(f"Function {func_idx+1}: {q_str}")


FileNotFoundError: [Errno 2] No such file or directory: 'inputs.txt'